# Парсинг автомобилей на Авито(линк, название, цена)

In [1]:
import re
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd 
import requests

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from tqdm import tqdm
from selenium.webdriver import ActionChains

In [3]:
def find_autos(brand, model):
    #Также в цикл можно вложить клик по номеру страницы для парсинга более одной страницы
    i=-10
    j = 0
    url = 'https://www.avito.ru/moskva/transport'
    browser = Chrome()  
    browser.get(url)
    arr=[]
    input_tab = browser.find_element(By.CLASS_NAME, 'input-input-Zpzc1')
    input_tab.send_keys(f'{brand} {model}')
    v = browser.find_element(By.CLASS_NAME, 'index-button-rtdlZ')
    v.click()
    sleep(2)
    q = browser.find_element(By.XPATH, '//a[@title="Автомобили"]')  
    try:
        q.click()
    except:
        pass
    sleep(2)
    autos = bs(browser.page_source).findAll('div', class_='iva-item-root-_lk9K photo-slider-slider-S15A_ iva-item-list-rfgcH iva-item-redesign-rop6P iva-item-responsive-_lbhG items-item-My3ih items-listItem-Gd1jN js-catalog-item-enum')
    sleep(2)
    #Условие для ограничения количества спаршенных авто; по необходимости его можно двигать
    while len(arr) >= i :
        browser.execute_script("window.scrollBy(0,300)")    
        try:
            name = autos[j].find('h3', class_='styles-module-root-GKtmM styles-module-root-YczkZ styles-module-size_l-iNNq9 styles-module-size_l_compensated-KFJud styles-module-size_l-YMQUP styles-module-ellipsis-a2Uq1 styles-module-weight_bold-jDthB stylesMarningNormal-module-root-S7NIr stylesMarningNormal-module-header-l-iFKq3').text
        except:
            name='none'
        try:
            link = autos[j].find('a').get('href')
        except:
            link = 'none'
        try:
            price = autos[j].find('strong', class_='styles-module-root-bLKnd').text
        except:
            price = 'none'
        if [link, name, price] not in arr and bool(re.search('moskva', link))==True:
            arr.append([link, name, price])
        i+=1
        j+=1
        sleep(3)
    return pd.DataFrame(arr, columns = ['link', 'name', 'price'])

In [5]:
find_autos('Audi', '80')

,link,name,price
0,/moskva/avtomobili/audi_80_2.0_mt_1990_400_000...,"Audi 80 2.0 MT, 1990, 400 000 км",145 000 ₽
1,/moskva/avtomobili/audi_80_2.0_at_1992_200_000...,"Audi 80 2.0 AT, 1992, 200 000 км",330 000 ₽
2,/moskva/avtomobili/audi_80_1.8_at_1990_320_000...,"Audi 80 1.8 AT, 1990, 320 000 км",240 000 ₽
3,/moskva/avtomobili/audi_80_2.0_mt_1992_360_000...,"Audi 80 2.0 MT, 1992, 360 000 км",350 000 ₽
4,/moskva/avtomobili/audi_80_1.8_at_1988_300_000...,"Audi 80 1.8 AT, 1988, 300 000 км",60 000 ₽
5,/moskva/avtomobili/audi_80_1.8_mt_1987_309_000...,"Audi 80 1.8 MT, 1987, 309 000 км",110 000 ₽
6,/moskva/avtomobili/audi_80_2.0_mt_1994_350_000...,"Audi 80 2.0 MT, 1994, 350 000 км",333 000 ₽
7,/moskva/avtomobili/audi_80_1.3_mt_1978_bityy_1...,"Audi 80 1.3 MT, 1978, битый, 10 000 км",14 560 ₽
8,/moskva/avtomobili/audi_80_1.6_mt_1990_220_000...,"Audi 80 1.6 MT, 1990, 220 000 км",235 000 ₽
9,/moskva/avtomobili/audi_80_1.8_mt_1989_277_000...,"Audi 80 1.8 MT, 1989, 277 000 км",170 000 ₽
